<a href="https://colab.research.google.com/github/mmistroni/OpenBB_Experiments/blob/main/openbbPlatformAsLLMTools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenBB Tools

## Getting Started

### Install dependencies
####https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/orchestration/langchain/intro_langchain_palm_api.ipynb#scrollTo=eVpPcvsrkzCk

In [ ]:
!pip install openbb
!pip install openbb-charting

!pip install pydantic

!pip install langchain
!pip install langchain_community
#!pip install langchain_openai
!pip install langchain-google-vertexai
# need to restart runtime after installing pydantic

### Set the VertexAI Key

In [ ]:
import google.generativeai as genai
from google.api_core import retry
from google.colab import userdata
gkey = userdata.get('AI_STUDIO_KEY')
pat = userdata.get('PAT')
genai.configure(api_key=gkey)
import vertexai
from google.colab import auth
PROJECT_ID = "datascience-projects"  # @param {type:"string"}
# Set the project id
#! gcloud config set project {PROJECT_ID}
#auth.authenticate_user()
vertexai.init(
    project=PROJECT_ID,
    location="us-central1",
    staging_bucket="gs://mm_dataflow_bucket",
)


### Import packages

In [ ]:
from openbb import obb

Extensions to add: openbb_charting@2.1.3

Building...


### Login

In [ ]:
obb.account.login(pat=pat)

/usr/local/lib/python3.10/dist-packages/openbb_core/app/service/hub_service.py:241: UserWarning: 
Deprecated v3 credentials found.

'API_FRED_KEY' -> 'FRED_API_KEY', 
'API_KEY_FINANCIALMODELINGPREP' -> 'FMP_API_KEY'

You can update them at https://my.openbb.co/app/platform/credentials.
  warn(


## OpenBB tool pre-requisites

### Callable

In [ ]:
obb.equity.price.historical(symbol="AAPL", provider="fmp").to_df()

,open,high,low,close,volume,vwap,adj_close,unadjusted_volume,change,change_percent
date,,,,,,,,,,
2023-06-21,184.90,185.41,182.59,183.96,49515697,184.2150,182.99,49515697.0,-0.94,-0.005084
2023-06-22,183.74,187.05,183.67,187.00,51245327,185.3650,186.01,51245327.0,3.26,0.017700
2023-06-23,185.55,187.56,185.01,186.68,53116996,186.2000,185.69,53116996.0,1.13,0.006090
2023-06-26,186.83,188.05,185.23,185.27,48088661,186.3450,184.29,48088661.0,-1.56,-0.008350
2023-06-27,185.89,188.39,185.67,188.06,50730846,187.0025,187.07,50730846.0,2.17,0.011700
...,...,...,...,...,...,...,...,...,...,...
2024-06-14,213.85,215.17,211.30,212.49,70122748,213.2025,212.49,70122748.0,-1.36,-0.006360
2024-06-17,213.37,218.95,212.72,216.67,93728300,215.4275,216.67,93728300.0,3.30,0.015500
2024-06-18,217.59,218.63,213.00,214.29,79943254,215.8775,214.29,79943254.0,-3.30,-0.015200


In [ ]:
obb.equity.price.historical("AAPL", start_date="2022-01-01", provider='fmp')

OBBject

id: 06675e4f-96c2-718a-8000-0028b5ce5d0f
results: [{'date': datetime.date(2022, 1, 3), 'open': 177.83, 'high': 182.88, 'low'...
provider: fmp
warnings: None
chart: None
extra: {'metadata': {'arguments': {'provider_choices': {'provider': 'fmp'}, 'standa...

### Input Schema

In [ ]:
obb.coverage.command_model[".equity.price.historical"]["openbb"]["QueryParams"]

{'fields': {'symbol': FieldInfo(annotation=str, required=True, description='Symbol to get data for.', json_schema_extra={'fmp': {'multiple_items_allowed': True}, 'polygon': {'multiple_items_allowed': True}, 'tiingo': {'multiple_items_allowed': True}, 'yfinance': {'multiple_items_allowed': True}}),
  'interval': FieldInfo(annotation=Union[str, NoneType], required=False, default='1d', description='Time interval of the data to return.'),
  'start_date': FieldInfo(annotation=Union[date, NoneType], required=False, default=None, description='Start date of the data, in YYYY-MM-DD format.'),
  'end_date': FieldInfo(annotation=Union[date, NoneType], required=False, default=None, description='End date of the data, in YYYY-MM-DD format.')},
 'docstring': 'Equity Historical Price Query.'}

In [ ]:
obb.coverage.command_model[".equity.price.historical"]["polygon"]["QueryParams"]

{'fields': {'interval': FieldInfo(annotation=str, required=False, default='1d', description='Time interval of the data to return. The numeric portion of the interval can be any positive integer. The letter portion can be one of the following: s, m, h, d, W, M, Q, Y'),
  'adjustment': FieldInfo(annotation=Literal['splits_only', 'unadjusted'], required=False, default='splits_only', description='The adjustment factor to apply. Default is splits only.'),
  'extended_hours': FieldInfo(annotation=bool, required=False, default=False, description='Include Pre and Post market data.'),
  'sort': FieldInfo(annotation=Literal['asc', 'desc'], required=False, default='asc', description="Sort order of the data. This impacts the results in combination with the 'limit' parameter. The results are always returned in ascending order by date."),
  'limit': FieldInfo(annotation=int, required=False, default=49999, description='The number of data entries to return.', metadata=[Gt(gt=0)])},
 'docstring': 'Poly

### Documentation

In [ ]:
help(obb.equity.price.historical)

Help on method historical in module openbb.package.equity_price:

historical(symbol: Annotated[Union[str, List[str]], OpenBBField(description='Symbol to get data for. Multiple comma separated items allowed for provider(s): fmp, polygon, tiingo, yfinance.')], interval: Annotated[Optional[str], OpenBBField(description='Time interval of the data to return.')] = '1d', start_date: Annotated[Union[datetime.date, NoneType, str], OpenBBField(description='Start date of the data, in YYYY-MM-DD format.')] = None, end_date: Annotated[Union[datetime.date, NoneType, str], OpenBBField(description='End date of the data, in YYYY-MM-DD format.')] = None, chart: typing.Annotated[bool, OpenBBField(description='Whether to create a chart or not, by default False.')] = False, provider: Annotated[Optional[Literal['fmp', 'intrinio', 'polygon', 'tiingo', 'yfinance']], OpenBBField(description='The provider to use, by default None. If None, the priority list configured in the settings is used. Default priority: f

## OpenBB tool

In [ ]:
from langchain_core.tools import StructuredTool

llm_historical_price = StructuredTool.from_function(
    func=obb.equity.price.historical,
    description=obb.equity.price.historical.__doc__.split('\n')[0]  # Use first line of docstring
)

## Multiple OpenBB Tools

In [ ]:
llm_tools = [
    StructuredTool.from_function(
        name=name,
        func=schema['callable'],
        description=schema['callable'].__doc__.split('\n')[0]
    )
    for name, schema
    in obb.coverage.command_schemas().items()
]

## OpenBB Tools fed to agent

In [ ]:
from langchain.agents import AgentExecutor, create_openai_functions_agent, create_tool_calling_agent
from langchain_google_vertexai import VertexAI, ChatVertexAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a very powerful assistant, but don't know current events"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

llm_tools = [
    StructuredTool.from_function(
        func=obb.equity.price.quote,
        description=obb.equity.price.quote.__doc__.split("\n")[0]
    )
]

#llm = VertexAI(model_name="gemini-pro")
llm = ChatVertexAI(model="gemini-pro")


In [ ]:
from langchain.schema import HumanMessage, SystemMessage
from langchain.llms import VertexAI
from langchain.embeddings import VertexAIEmbeddings
from langchain.chat_models import ChatVertexAI
from google.cloud import aiplatform
import time
from typing import List

# LangChain
import langchain
from pydantic import BaseModel

print(f"LangChain version: {langchain.__version__}")

# Vertex AI

print(f"Vertex AI SDK version: {aiplatform.__version__}")

LangChain version: 0.2.5
Vertex AI SDK version: 1.56.0


In [ ]:
agent = create_tool_calling_agent(llm=llm, tools=llm_tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=llm_tools, verbose=True)

agent_executor.invoke({"input": "What is the latest stock price of AAPL?"})



> Entering new AgentExecutor chain...


InvalidArgument: 400 Unable to submit request because one or more function parameters didn't specify the schema type field. Learn more: https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling